In [ ]:
import {NeuroModule, nm, BinLayer, Linear} from '/web/oda/apps/neuronet/genius/neuro-module/neuro-module.js';
import {tensor} from '/web/oda/apps/neuronet/genius/torus/torus.js';

In [ ]:
MNIST = class MNIST  extends NeuroModule{
    constructor(dim = 28) {
        super(arguments);
    }
    
    __init__(){
        this.layer1 = new Linear(this.dim*this.dim,this.dim*this.dim/2,true)
        this.layer2 = new Linear(this.dim*this.dim/2,10,true)
    }
    
    forward(x){
        x=tensor.from(x)
        x = this.layer1(x)
        x = x.sigm()
        x = this.layer2(x)
        x = x.sigm()
        return x
    }
}


In [ ]:
// model = await this.contextItem.owner.getFile('mnist/model.json');
// out_size = 10
//  if(model){
//     //  model = await model.load();
//      mnist = new MNIST(model);
//  }
//  else
mnist = new MNIST();
//loss.back();
//this.contextItem.$owner.saveFile(mnist.toJSON(), 'mnist/model.json')
print(mnist)

{
    "$": "MNIST",
    "dim": 28,
    "prototype": {}
}

In [ ]:
datasetRaw = await ODA.loadJSON('/web/oda/apps/neuronet/mnist/mnist.json');

getTrain = (dataset = datasetRaw, batch=5, test_size=0.2) => {
    let [result, res] = [{inputs:[],targets:[]},[]] 

    let suBatch = Array(10).fill(0).map((_,i)=>i) //.sort(()=>Math.random() - 0.5)
    suBatch.forEach(n=>{
        let target  = Array(10).fill(0);
        target[n] = 1
        for (let j=0; j<batch; j++) {
            let br = Math.floor(dataset[n].length * (1-test_size))
            let k = Math.floor(Math.random() * br)
            res.push([dataset[n][k].flat(),target])
        }
    })
        
    res.sort(()=>Math.random() - 0.5)
    res.forEach ( ([inp,tar]) => {result.inputs.push(inp); result.targets.push(tar)} )
    return result
}

// print(datasetRaw[0][0].flat())

{}

In [ ]:
// dat = mnist.getTrain(datasetRaw , 1)
// print(`getTrain возвращает { inputs (${dat.inputs.length} по ${dat.inputs[0].length}) ,targets (${dat.targets.length} по ${dat.targets[0].length}) }.`)

// let loss='',rez=''

// // print(dat.inputs[0])

// for (let i=0; i<dat.inputs.length; i++) {
//     // print (i)
//     rez = mnist.forward(dat.inputs[i])
//     loss = rez.crossEntropy (dat.targets[i]);
//     loss.back();
// }
// print (loss)

In [ ]:
// dat = mnist.getTrain(datasetRaw , 1)
// print(`getTrain возвращает { inputs (${dat.inputs.length} по ${dat.inputs[0].length}) ,targets (${dat.targets.length} по ${dat.targets[0].length}) }.`)

// let loss='',rez=''

// // print(dat.inputs[0])

// for (let i=0; i<dat.inputs.length; i++) {
//     // print (i)
//     rez = mnist.forward(dat.inputs[i])
//     loss = rez.crossEntropy (dat.targets[i]);
//     loss.back();
// }
// print (loss)

In [ ]:
ODA({is: 'oda-mnist', imports: '@oda/button', template: /*html*/ `
    <style>
        :host {max-width:1000px; margin:auto; display:grid; grid-template-columns:1fr 1fr; grid-template-areas: 'input output' 'loss loss'; }
        .block {border: 1px solid #4d85cf; padding:10px; margin:10px;}
        .box { margin:10px}
        .block h3 {margin:0 10px;}
        oda-button {border: 1px solid #4d85cf; margin:10px;}}
    </style>
    <div class='block' style='grid-area:input;'>
        <h3>input:</h3>
        <svg class='box' width="281" height="281" xmlns='http://www.w3.org/2000/svg' viewBox='0 0 281 281'>
            <rect x="0" y="0" width="281" height="281" fill='#4d85cf'/>
            <rect ~for='rects' :x="$for.item?.x" :y="$for.item?.y" width="9" height="9" :fill='$for.item?.v?"#070637":"#a9caff"'/>            
        </svg>
    </div>
    <div class='block' style='grid-area:output;'>
        <h3>output:</h3>
        <svg class='box' width="281" height="281" xmlns='http://www.w3.org/2000/svg' viewBox='0 0 281 281'>
            <rect x="0" y="0" width="281" height="281" fill='#a9caff'/>
            <g ~for='result' :transform="'translate(0,'+ ($for.index * 28) +')'">
                <rect x="2" y="2" :width="276*$for.item" height="24" :fill='maxIndex===$for.index? "#ff00ff55" : "#4d85cf55"'/>
                <text x="265" y="16" font-size='16' dominant-baseline="middle" fill="#070637">{{$for.index}}</text>
                <text x="5" y="16" font-size='16' dominant-baseline="middle" fill="#070637">{{($for.item*100).toFixed(2)}} %</text>
            </g>
            <text x="150" y="160" font-size='200' dominant-baseline="middle" text-anchor="middle" fill="#070637">{{maxIndex}}</text>                
        </svg>
    </div>
    <div class='block' style='grid-area:loss;'>
        <h3>loss:</h3>
        <svg class='box' :width :height xmlns='http://www.w3.org/2000/svg' :viewBox="'0 0 ' + width + ' ' + height">
            <rect x="0" y="0" :width :height fill='#a9caff'/>
            <path :d='pathD' fill="none" stroke="#070637" stroke-width="2"/>
        </svg>
    </div>
    <oda-button :label='go?"pause":"play"' :icon='go?"av:pause":"av:play-arrow"' @tap="_go()"></oda-button> 
    `,
    get rects() {
        return this.input.map((v,i)=> {
            let [x,y] = [i%28, ~~(i/28)].map(a=>a*10+1)
            return {x,y,v}
        } )
    },
    get maxIndex() { return this.result.reduce((a,b,i,c)=>c[a]>b?a:i,0) },

    get pathD(){
        if (this.pointsL.length<2) return ''
        let padding = Math.min(this.width,this.height)/10
        let [w,h] = [this.width,this.height].map(x=>x-padding*2)
        let dx = w/(this.pointsL.length-1)
        let [min,max] = [Math.min(...this.pointsL),Math.max(...this.pointsL)]
        let points = this.pointsL.map((v,i)=>[i*dx+padding,h-(v-min)/(max-min)*h+padding])
        let rez = 'M ' + points[0].join(' ') + ' '
        if (this.bezier) rez+=  'Q ' + points[0].join(' ') + ', ' + points[1].join(' ') + ' ' 
        else  rez += 'L ' + points[1].join(' ') + ' '
        points.slice(2).forEach(e => rez += (this.bezier?'T ':'L ') + e.join(' ') + ' ' )
        return rez
    },

    get pointsL() {
        let [lL,lP] = [this.loss.length,this.maxPoint]
        if (lL<lP) return this.loss
        let newP = Array(lP-1).fill(0).map((_,i)=>{
            let idx = Math.floor(i*(lL/(lP-1)))
            let dL = Math.floor(lL/(lP-1))+1
            return this.loss.slice(idx,idx+dL).reduce((a,c)=>a+c)/dL
        })
        newP.push(this.loss[lL-1])
        return newP
    },

    go:false,
    _go(){
        this.go = !this.go
        this.steps()
    },

    width:281*2+10*6+2,
    height:281,
    maxPoint:100,

    input: {
        $def: new Array(28*28).fill(0)
    },
    result:{
        $def: new Array(10).fill(0.1)
    },
    loss:{
        $def:[]//new Array(20000).fill(0).map(_=> Math.random()*5-2)
    },

    mnist: new MNIST(),
    getTrain:getTrain,

    step() {
        let dat = this.getTrain(undefined, 1)
        let loss, rez;
        for (let i=0; i<dat.inputs.length; i++) {
            rez = this.mnist(dat.inputs[i])
            loss = rez.crossEntropy (dat.targets[i]);
            this.loss.push(loss.data[0])
            loss.back();
            if (i===0) {
                this.input = dat.inputs[i]
                this.result = rez.data
            }
            // console.log(loss.data[0])
        }
    },
    steps(){
        if (this.go) setTimeout(() => {
            this.step()
            this.steps()
        }, 0)            
    }

})


In [ ]:
layer1 = nm.linear(10, 10);
layer2 = nm.linear(10, 10);
print(`layer1 (${layer1.dim_in} x ${layer1.dim_out}), layer2 (${layer2.dim_in} x ${layer2.dim_out})`)

// layer3 = new BinLayer(input.length, target.length);

layer1 (undefined x undefined), layer2 (undefined x undefined)

In [ ]:
// print(tensor.LEARNING_RATE)
tensor.LEARNING_RATE = 0.3;
// print(tensor.LEARNING_RATE)
loss = 0
for (let i=0; i<10; i++) {
    input = Array(10).fill(0)
    input[Math.floor(Math.random()*10)]=1
    // print(input.join(''))
    
    result = layer1(input);
    // print('layer1', result);
    result = result.sigm();
    // print('sigm1', result);
    result = layer2(result);
    // print('layer2', result.data);
    result = result.sigm();
    
    result = result.softmax();
    
    
    
    let res = ''
    result.data.forEach(x=> res += (' ' + x.toFixed(1)) )
    // print(res)

    print(` input = ${input.join('   ')} \n result=${res} \n max=    ${softmax(result.data).join('   ')}` )
    // print('sigm2',, ,  ;
    
    loss = result.crossEntropy (input);
    // print('loss', loss.data[0]);
    loss.back();
}
print('loss', loss.data[0]);


print([...result.data])


// i = 0 

ReferenceError:
softmax is not defined
(27:54)

In [ ]:

// if (i<dat.targets.length) {
//     res = mnist(dat.inputs[i])
//     loss = res.MSE(dat.targets[i])
//     print(`subStep= ${i} target=${dat.targets[i].join('')} res=${res.data} `)
//     print('loss',loss.data[0])
//     loss.back()
//     i += 1
// }
 


// loss = res.MSE(target)

// print(mnist.layer1.WEIGHTS.data.toString())
// res1 = mnist(input)
// loss1 = res.MSE(target)
// print(res1.data)
// print(loss1.data)
// res2 = mnist(input)
// loss2 = res.MSE(target)
// print(res2.data)
// print(loss2.data)

In [ ]:
function binarize (grad_i) {
    let p = Math.max(0,Math.min(1,(grad_i+1)/2)) // σ(x) = hard sigmoid
    return p>Math.random() // +1 with probability p = σ(x), -1 otherwise.
}

let grad = -0.7
print (`σ(x) = ${Math.max(0,Math.min(1,(grad+1)/2))} = ${Math.max(0,Math.min(1,(grad+1)/2))*100} % ` )

let n = 1000
let rezz = Array(n).fill(grad).map(g => binarize(g)).reduce((akk,b) => { b? akk.true++:akk.false++; return akk },{true:0,false:0} )

print(`true: ${rezz.true}, (${rezz.true/n*100} %) `)

σ(x) = 0.15000000000000002 = 15.000000000000002 % 

true: 159, (15.9 %) 

In [ ]:
v = Array(15).fill(0).map(a =>  (Math.random()>0.5)?0:1  ) // [0,1,1,1,0]
mul = v.filter(b=>b).reduce((akk,b) => akk*(-1), 1 )>0
print (mul)
result = 'return (' + v.map((_,i)=>{
                    return '(v['+i+'] - 1)';
            }).filter(l=>l).join(` + `) + ')&1;\n';
print(result)
const fn = new Function('v',result);
print(fn(v))
// print(`in= ${v.join('')}  mul =`fn(v))

true

return ((v[0] - 1) + (v[1] - 1) + (v[2] - 1) + (v[3] - 1) + (v[4] - 1) + (v[5] - 1) + (v[6] - 1) + (v[7] - 1) + (v[8] - 1) + (v[9] - 1) + (v[10] - 1) + (v[11] - 1) + (v[12] - 1) + (v[13] - 1) + (v[14] - 1))&1;


1

In [ ]:
    // print(`input= ${input.join('')} result = ${result.data.map(x=>(Math.round(x*10)/10))} ` )
    
print(`-1- ${Array(5).fill(0).map(y=>Math.random()).map(x=>(Math.round(x*10)/10))}) -1-`)

h = Array(5).fill(0).map(y=>Math.random())
print(`-2- ${h.map(x=>(Math.round(x*10)/10))}) -2-`)


-1- 0.8,0.9,0.6,0.7,1) -1-

-2- 0.8,0.7,0.2,0.5,0.3) -2-

In [ ]:
// let softmax = (output) => {
//     let maximum = output.reduce(function(p,c) { return p>c ? p : c; });
//     let nominators = output.map(function(e) { return Math.exp(e - maximum); });
//     let denominator = nominators.reduce(function (p, c) { return p + c; });
//     let softmax = nominators.map(function(e) { return e / denominator; });
//     let maxIndex = 0;
//     softmax.reduce(function(p,c,i){if(p<c) {maxIndex=i; return c;} else return p;});
//     let result = [];
//     for (var i=0; i<output.length; i++) (i==maxIndex)? result.push(1) : result.push(0)
//     return result;
// }

// let tt = Object.values({
//     "0": 0.09848282486200333,
//     "1": 0.12310634553432465,
//     "2": 0.09121625870466232,
//     "3": 0.12418807297945023,
//     "4": 0.11233071237802505,
//     "5": 0.09030517935752869,
//     "6": 0.08965815603733063,
//     "7": 0.0996328815817833,
//     "8": 0.08942568302154541,
//     "9": 0.081653892993927
// })
// print(softmax([1,3,8]))

let softmax = (output) => {
    let exps = output.map(x=>Math.exp(x))
    let sum = exps.reduce((akk,v)=>akk+v)
    return exps.map(e=>e/sum)
}
print(softmax([1,3,8]))

x = tensor.from([1,3,8])

print(x.softmax())

[
    0.0009049591825877395,
    0.006686794167383233,
    0.992408246650029
]

{
    "$": "tensor",
    "shape": "3",
    "dType": "Float32Array",
    "data": "0.0009049591608345509 0.006686794105917215 0.992408275604248"
}

In [ ]:
xor = [
  { input: [0, 0], output: [0]},
  { input: [0, 1], output: [1]},
  { input: [1, 0], output: [1]},
  { input: [1, 1], output: [1]}];
  
layer_xor = nm.linear(2, 1, true);
tensor.LEARNING_RATE = 0.3;
loss = 0

In [ ]:

loss = 0
for (let i=0; i<1000; i++) {
    let x = xor[Math.floor(Math.random()*4)]
    
    // print (tensor.from(x.input) ) 

    result = layer_xor(tensor.from(x.input));

    // print(` input = ${x.input.join(',')}  result= ${result.data[0].toFixed(2)} ` )
    // print('sigm2',, ,  ;
    
    loss = result.MSE(x.output);
    // print('loss', loss.data[0]);
    loss.back();
}
print('loss', loss.data[0]);

loss
0.1590086966753006

In [ ]:

loss = 0
for (let i=0; i<1000; i++) {
    let x = xor[Math.floor(Math.random()*4)]
    
    // print (tensor.from(x.input) ) 

    result = layer_xor(tensor.from(x.input));

    // print(` input = ${x.input.join(',')}  result= ${result.data[0].toFixed(2)} ` )
    // print('sigm2',, ,  ;
    
    loss = result.MSE(x.output);
    // print('loss', loss.data[0]);
    loss.back();
}
print('loss', loss.data[0]);

 input = 0,0  result= 0.29 

 input = 0,1  result= 0.96 

 input = 1,0  result= 0.84 

 input = 1,1  result= 1.51 